In [1]:
import json
from sentence_transformers import SentenceTransformer

# Modeli yükle
model = SentenceTransformer('paraphrase-MiniLM-L6-v2')  # Bu model küçük ve hızlıdır


In [40]:
import json

# JSON dosyasını oku
with open(r'datasets\faq_data.json', 'r', encoding='utf-8') as f:
    data = json.load(f)

# Elasticsearch formatında veriyi düzenle
data_to_index = []  # Elasticsearch'e yüklemek için veri saklama listesi
i = 0

# Her bir veri için embedding al
for item in data:
    if i < 100:
        # Question ve Answer'ı birleştir
        combined_text = f"{item['question']} {item['answer']}"
        
        # Metnin embedding'ini al
        embedding = model.encode(combined_text).tolist()  # Embedding'i al ve listeye çevir
        
        # Kategori bilgisi (örneğin 'category' alanı varsa)
        category = item.get('category', 'default')  # 'category' alanı yoksa varsayılan 'default' değeri
        
        # Elasticsearch formatında veri düzenle
        document = {
            'embedding': embedding,
            'text': combined_text,
            'category': category
        }
        
        # Veriyi listeye ekle
        data_to_index.append(document)
        i += 1

# Sonuçları yazdırmak (isteğe bağlı)
print(data_to_index)

[{'embedding': [0.2426999807357788, 0.3390394151210785, -0.10511984676122665, -0.1706264615058899, -0.18055839836597443, -0.5007614493370056, 0.5554322004318237, -0.2882031500339508, 0.018552936613559723, -0.18429049849510193, 0.5355144143104553, -0.5217718482017517, 0.21282650530338287, -0.12151714414358139, -0.007502521388232708, -0.27454057335853577, -0.003108300268650055, 0.6535873413085938, 0.10683579742908478, -0.0576932355761528, -0.6606277823448181, -0.25982022285461426, 0.13520987331867218, 0.08888298273086548, 0.5280755162239075, -0.26848524808883667, 0.1197432354092598, 0.04437793046236038, -0.3786744177341461, -0.02484360709786415, 0.17896044254302979, 0.3580542802810669, 0.2657201886177063, 0.2891175448894501, 0.18097350001335144, 0.4696822166442871, -0.3038792610168457, 0.455569863319397, 0.15367268025875092, 0.1768355369567871, -0.017375007271766663, -0.6165407299995422, -0.026032833382487297, -0.34260788559913635, -0.029976144433021545, -0.09689038246870041, 0.127068102

In [3]:
from elasticsearch import Elasticsearch
from elasticsearch import helpers

In [4]:
# Create the Elasticsearch client.
def get_client(hosts: list, user: str = None, password: str = None):
    if user and password:
        return Elasticsearch(hosts, http_auth=(user, password), verify_certs=False, ssl_show_warn=False)
    else:
        return Elasticsearch(hosts)

In [76]:
# For a security cluster with HTTPS enabled, run the following:
es_client = get_client(hosts=['https://159.138.111.237:9200'], user='admin', password='Huawei@1234.')

In [88]:
# Create an index table.
def create(client: Elasticsearch, index: str):
    # Index mapping information
    index_mapping = {
        "settings": {
            "index": {
                "vector": "true",  # Enable the vector feature.
                "number_of_shards": 1,  # Set the number of index shards as needed.
                "number_of_replicas": 0,  # Set the number of index replicas as needed.
            }
        },
        "mappings": {
            "properties": {
                "embedding": {
                    "type": "vector",
                    "dimension": 384,
                    "indexing": True,
                    "algorithm": "GRAPH",
                    "metric": "euclidean"
                },
                "text": {
                    "type": "text"
                },
                "category": {
                    "type": "keyword"
                }
            }
        }
    }
    res = client.indices.create(index=index, body=index_mapping)
    print("create index result: ", res)

In [90]:
# Test the index name.
index_name = "hakanindex4"

# Create an index.
create(es_client, index=index_name)

create index result:  {'acknowledged': True, 'shards_acknowledged': True, 'index': 'hakanindex4'}


In [72]:
from elasticsearch import helpers

def write(client: Elasticsearch, index: str, vecs: list, bulk_size=500):
    for i in range(0, len(vecs), bulk_size):
        actions = [
            {
                "_op_type": "index",  # "index" to either create or update
                "_index": index,
                "_source": {
                    "embedding": vec['embedding'],
                    "text": vec['text'],
                    "category": vec['category']
                }
            }
            for vec in vecs[i: i + bulk_size]
        ]
        
        try:
            # Perform bulk indexing
            success, errors = helpers.bulk(client, actions, request_timeout=3600)

            if errors:
                print(f"Some documents failed to index: {errors}")
            else:
                print(f"Successfully indexed {success} documents in batch {i//bulk_size + 1}")

        except Exception as e:
            print(f"Error during bulk indexing: {str(e)}")

    try:
        # Refresh the index to ensure that the newly indexed documents are searchable
        client.indices.refresh(index=index, request_timeout=3600)
        print(f"Index '{index}' refreshed successfully.")
    except Exception as e:
        print(f"Failed to refresh index. Error: {str(e)}")


In [91]:
index_name = "hakanindex4"
# Write data.
write(es_client, index=index_name, vecs=data_to_index)

Successfully indexed 83 documents in batch 1
Index 'hakanindex4' refreshed successfully.


In [97]:
def search(client: Elasticsearch, index: str, query: list, size: int):
    query_body = {
        "size": size,
        "query": {
            "vector": {
                "embedding": {
                    "vector": query,
                    "topk": size
                }
            }
        }
    }
    
    res = client.search(index=index, body=query_body)
    return res  # Return extracted data

In [123]:
query_vector = search(es_client, index=index_name, query=model.encode('GetirAraç saatlik, dakikalık veya günlük araç kullanım hizmetidir. GetirAraç’ı kullanarak sana en yakın aracı uygulama üzerinden, ihtiyacın olan süre kadar kullanabilirsin. Kullandığın aracı istediğin yerde, dilediğin kadar kullandıktan sonra hizmet bölgesi içindeki herhangi bir yere bırakabilirsin.').tolist(), size=10)

In [124]:
for result in query_vector["hits"]["hits"]:
    print(f"Score: {result['_score']} | Text: {result['_source']['text']}")


Score: 1.0 | Text: GetirAraç nedir? GetirAraç saatlik, dakikalık veya günlük araç kullanım hizmetidir. GetirAraç’ı kullanarak sana en yakın aracı uygulama üzerinden, ihtiyacın olan süre kadar kullanabilirsin. Kullandığın aracı istediğin yerde, dilediğin kadar kullandıktan sonra hizmet bölgesi içindeki herhangi bir yere bırakabilirsin.
Score: 0.18165587 | Text: GetirAraç'ın temizlik tanımı nedir? GetirAraç olarak temizlik bizim için çok önemli. Araçlarımız, gün içinde pek çok kullanıcıya hizmet veriyor. Bundan dolayı bazen müdahale gerektirmeyen tozlanmalar olabiliyor. Fakat araç içinde çamur, kül ya da farklı bir kirlilik oluşturan durumlarla karşılaşırsan, bizimle paylaş. Verdiğin geri bildirimler doğrultusunda saha ekibimizi hemen ilgili araca yönlendirelim.
Score: 0.15883157 | Text: GetirAraç'ı kimler kullanabilir? Genel kullanım koşullarımıza göre, GetirAraç'ı kullanabilmek için yeni üyeliklerde- 21-25 yaş arası kullanıcılarda en az 2 yıllık ve üzerinde TCKN bulunan ehliyet ve kred

TransportError: TransportError(500, 'search_phase_execution_exception', 'java.lang.NullPointerException: Cannot invoke "org.apache.lucene.index.FieldInfo.getAttribute(String)" because "fieldInfo" is null')

Score: 0.047874957 | Text: Paket tutarı dışındaki tutarlar da taksite dahil mi? Köprü ve otoyol geçiş ücretleri ve paket aşım tutarı taksitlendirilmez.
Score: 0.047600158 | Text: Rezervasyonlu yaptığım kiralamada otopark ücreti ödeyecek miyim? Ücretli otoparklardan kullanımını başlattığında ödemesini yapar, kullanımının sonunda 4 açı fotoğraftan sonra göreceğin ‘’Otopark Fişi’' sekmesinden fotoğrafını paylaşırsın. 2 gün içinde ödediğin tutarın %50 fazlasını sana kampanya olarak tanımlarız.

İzmir’de ise İzelman otoparkları ücretsiz olarak kullanılmaktadır, araç alırken de bırakırken de ödeme yapman gerekmez.
Uygulamanın sağ üst köşesindeki “Otoparklar” ikonundan ücretsiz olarak kullanabileceğin otoparkları görebilirsin.
Score: 0.046872735 | Text: Kullandığım aracın plakasına trafik cezası kesildi? Ben mi ödeyeceğim? Kullanımın sırasında trafik cezası kesilmesi durumunda KABİS aracılığıyla ceza tutanağı, direkt senin adına düzenlenir. Böyle bir durumda cezanı, E-Devlet üzerinden kontrol

In [35]:
# Delete an index.
def delete(client: Elasticsearch, index: str):
    res = client.indices.delete(index=index)
    print("delete index result: ", res)

In [36]:
delete(es_client, index='qa_index')

delete index result:  {'acknowledged': True}
